<a href="https://colab.research.google.com/github/EP111111/Doc_analysis/blob/main/docs_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth as google_auth
google_auth.authenticate_user()


In [ ]:
!pip install "shapely<2.0.0"
!pip install google-cloud-aiplatform --upgrade
!pip install textract
!pip install python-docx

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel
from google.colab import drive
import textract
import numpy as np
import pandas as pd
import re


In [ ]:
PROJECT_ID = "ics-analysis-dev"
vertexai.init(project=PROJECT_ID, location="us-central1")

model = TextGenerationModel.from_pretrained("text-bison@001")

In [ ]:
from functions import *
import docx

# Read long docs to be processed

In [ ]:
def read_word_file(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text


document_text = read_word_file("Project Alpha.docx")



# Seperating docs into sections
Palm2 has a limit of 8000 tokens
if the section is long, we will have less sections, then the model will be faster.
But if the section is too long, the output will exceed the maxmium output size which is 1024 token
if the section is short, we will call palm2 more times. it takes longer time

In [ ]:
LLM_c= LLM_SPLIT_FUNC(document_text,model,13500)
LLM_c.keys()

dict_keys([0, 1, 2])

In [ ]:
length=0
for i in LLM_c.values():
  print(len(i))
  length+=len(i)
print(f'total length： {length}')
print(f'document length： {len(document_text)}')

10539
12020
13227
total length： 35786
document length： 35786


In [ ]:
#print seperator
for i in LLM_c.values():
  if i.split('\n')[0]=='':
    print(i.split('\n')[1])
  else:
    print(i.split('\n')[0])


The Energy Program at AGENTS is there to assist members of the community with their heating costs throughout the winter as well as home electricity needs. This is a program that relies heavily on specific state and federal regulations and funding sources, so it must be managed and reconfigured on a yearly basis as the regulations and funding sources change.
VENDOR PROPOSAL


In [ ]:
#generate questions for each section
questions={}
for ind, i in enumerate(LLM_c.keys()):
  a=Query_Vertex('take questions out from the provided article, if no question in the article, return "no". ' ,LLM_c[i],model).split('\n')
  if len(a)!=1:
    print(i)
    print(a)
    questions[i]=a

0
['Are there any specific requirements for the software?', 'What are the technical specifications of the system?', 'What are the reporting requirements?', 'What are the data security requirements?', 'What are the accounting requirements?']
1
['Provide an overview of your company, including date of incorporation, operating locations, number of employees, area(s) of specialty and number of successful projects with Non-Profit / Community Action Program customers.', 'Are you a woman or minority owned business? Please indicate in your response if you are.', 'Provide information about the specification of the project organizations with which you have experience (e.g., onshore, offshore, onshore PM, offshore dedicated development team, etc.).', 'Describe how you screen and hire staff. What level of skill and certifications, if any, must they have? How do you evaluate their experience and performance? What was your staff turnover rate throughout the preceding three (3) years for each year? Do